In [1]:
#Getting HTML

import requests
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
import time 
import re

url = "https://www.titan.fitness/strength/weight-plates/cast-iron-plates/cast-iron-olympic-plates/CPLATE_GROUP.html"
url = "https://yorkbarbell.com/product/2-inch-cast-iron-olympic-plate/"
url = "https://www.roguefitness.com/rogue-olympic-plates?gclid=Cj0KCQiAlZH_BRCgARIsAAZHSBlVDa0HIv_X2ChS5SpidgShkRbWHE4rAtYiChwfHKxorzGCDMQAu7gaAjlXEALw_wcB"
r = requests.get(url)

print(r.content[:500])

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n    <meta http-equiv="Content-type" content="text/html;charset=UTF-8"/>\r\n    <meta id="metaViewport" content="width=device-width" name="viewport"/>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=11"/>\r\n\r\n    \r\n\r\n    <style type="text/css">@font-face{font-family:Roboto Condensed;font-style:normal;font-weight:400;src:local("Roboto Condensed"),local("RobotoCondensed-Regular"),url(data:font/woff2;base64,d09GMgABAAAAACk8AA4AAAAAT4QAACjjAAEAAAAAAAAAAAAAA'


In [3]:
#Saving HTML Locally

def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)
        
        
save_html(r.content, 'RogueCastIron')


In [4]:
#Opening HTML from local file
    
def open_html(path):
    with open(path, 'rb') as f:
        return f.read()
    
    
html = open_html("RogueCastIron")  

In [39]:
#Making BeautifulSoup Object

from bs4 import BeautifulSoup
import lxml

soup = BeautifulSoup(r.content, 'html.parser')

In [23]:
#Plate class in order to store the data for each plate

class plateInfo:
    def __init__(self, company, link, weight, price, stock):
        self._company = company
        self._link = link
        self._weight = weight
        self._price = price
        self._stock = stock
        self._display = (weight,price,stock)
    
    def getInfo(self):
        return list(self._display)

In [44]:
#Extraction for Titan Fitness

company = "Titan Fitness"
plates = soup.findAll("div", class_="set-item")

header = ("Weight", "Price", "Stock")
print(list(header))

#Looping through all Plate listings
for plate in plates:
    #Link
    link = plate.select_one(".product-name a")["href"]
    link = "titan.fitness" + link
    
    #Price
    price = plate.find("span", class_="sup-hide").text.strip()
    price = float(price[1:])
    
    #Weight
    weightString = plate.find("h3", class_="product-name").text.strip()
    weight = []
    for t in weightString.split():
        try:
            weight.append(float(t))
        except ValueError:
            pass
    weight = weight[0]
    
    #Stock
    stock = plate.find("span", class_="availability-msg").text.strip()
    if "out" in stock:
        stock = "In Stock"
    else:
        stock = "Out of Stock"
        
    resultPlate = plateInfo(company, link, weight, price, stock)
    print(resultPlate.getInfo())


['Weight', 'Price', 'Stock']
[245.0, 419.99, 'Out of Stock']
[2.5, 29.99, 'Out of Stock']
[5.0, 29.99, 'Out of Stock']
[10.0, 39.99, 'Out of Stock']
[25.0, 74.99, 'Out of Stock']
[35.0, 54.99, 'Out of Stock']
[45.0, 64.99, 'Out of Stock']


In [24]:
#Extraction for York Barbell (using selenium)

company = "York Barbell"
link = "https://yorkbarbell.com/product/2-inch-cast-iron-olympic-plate/"

#Selenium accessing chrome broswer driver in incognito mode and without opening broswer window
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver", options=options)

#Iterating through the dropdown and automating clicking
driver.get("https://yorkbarbell.com/product/2-inch-cast-iron-olympic-plate/")
plates = driver.find_elements_by_class_name("attached")

for x in range(len(plates)):
        
        #Weight (Before click)
        weightString = plates[x].text.strip()
        weight = []
        for t in weightString.split():
            try:
                weight.append(float(t))
            except ValueError:
                pass
        weight = weight[0]
                
        #Clicks each dropdown option
        plates[x].click()
        time.sleep(1)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        
        #Price
        priceString = soup.select_one(".woocommerce-variation-price").text.strip()
        length = len(priceString)
        price = priceString[1:length]
        
        #Stock
        stock = soup.select_one(".woocommerce-variation-availability").text.strip()
        
        plates = driver.find_elements_by_class_name("attached")
        resultPlate = plateInfo(company, link, weight, price, stock)
        print(resultPlate.getInfo())
        
driver.close()



    #resultPlate = plateInfo(company, link, weight, price, stock)
    #print(resultPlate.getInfo())

NameError: name 'BeautifulSoup' is not defined

In [26]:
#Extraction for Rogue Fitness

company = "Rogue Fitness"
link = "https://www.roguefitness.com/rogue-olympic-plates?gclid=Cj0KCQiAlZH_BRCgARIsAAZHSBlVDa0HIv_X2ChS5SpidgShkRbWHE4rAtYiChwfHKxorzGCDMQAu7gaAjlXEALw_wcB"

#Selenium accessing chrome broswer driver in incognito mode and without opening broswer window
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/local/bin/chromedriver", options=options)

#Iterating through the dropdown and automating clicking
driver.get("https://www.roguefitness.com/rogue-olympic-plates?gclid=Cj0KCQiAlZH_BRCgARIsAAZHSBlVDa0HIv_X2ChS5SpidgShkRbWHE4rAtYiChwfHKxorzGCDMQAu7gaAjlXEALw_wcB")

weights = driver.find_elements_by_class_name("item-name")
prices = driver.find_elements_by_class_name("item-price")
stocks = driver.find_elements_by_class_name("add-to-cart")



for x in range(len(weights)):
    #Weight
    weightString = weights[x].text.strip()
    weight = []
    for t in weightString.split("LB"):
        try:
            weight.append(float(t))
        except ValueError:
            pass
    weight = weight[0]
    
    #Price
    price = prices[x].text.strip()
    price = price[1:len(price)]
    
    #Stock
    if(stocks[x].text.strip() == "Notify Me"):
        stock = "Out of Stock"
    else:
        stock = "In Stock"
    
    resultPlate = plateInfo(company, link, weight, price, stock)
    print(resultPlate.getInfo())
    


[1.25, '7.50', 'Out of Stock']
[2.5, '8.50', 'Out of Stock']
[5.0, '15.00', 'Out of Stock']
[10.0, '24.25', 'Out of Stock']
[25.0, '64.00', 'Out of Stock']
[35.0, '88.00', 'Out of Stock']
[45.0, '105.00', 'Out of Stock']
[100.0, '250.00', 'Out of Stock']


In [56]:
import tweepy as tp
import time
import os

APIKey = "Lhcnye2K3sKUiwimLwz0ET2qt"
APISecretKey = "5EVmjTS2Q59K2gnTg2kGHwpDbKRHwIMFDDXwYB4lnSjnXuQAfv"
AccessToken = "1334966865731448832-8k0oBUnUotF5oCKDMj8THsAOKoR4xh"
AccessTokenSecret = "3pGY3nSHhqVn9znwnaXojfKASTORiBWAitSPVWa155ZIh"
BearerToken = "AAAAAAAAAAAAAAAAAAAAAOtsKQEAAAAAyowOIGR2YX47m4XaTx0AeMy2s1g%3DzWKIDDcTXLCQlFEXrl4Rwe8gndWqyPMeGDeSMujqZ61s7iWf0G"

#Login
auth = tp.OAuthHandler(APIKey, APISecretKey)
auth.set_access_token(AccessToken, AccessTokenSecret)
api = tp.API(auth)

api.update_status("testing123")

Status(_api=<tweepy.api.API object at 0x7f8ade8a7d90>, _json={'created_at': 'Wed Dec 16 02:46:55 +0000 2020', 'id': 1339039274616938496, 'id_str': '1339039274616938496', 'text': 'testing123', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1334966865731448832, 'id_str': '1334966865731448832', 'name': 'Cast Iron Plate Price Tracker', 'screen_name': 'CastPrice', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Fri Dec 04 21:04:55 +0000 2020', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 2, 'lang': None, 'contributors_enabled': False, 'is_tran